In [2]:
import pandas as pd
# %pip install tensorflow[and-cuda]
import os

dir_notebook = os.getcwd()
print(dir_notebook)
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)
print("Directorio de trabajo actual:", os.getcwd())

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula
Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [3]:
df_precio = pd.read_parquet('data/processed/datos_precio/clima_precio_merged.parquet')

In [4]:
df_precio.head()

,timestamp,€/kwh,fecha,indicativo_3196,altitud_3196,tmed_3196,tmin_3196,tmax_3196,velmedia_3196,racha_3196,...,horaPresMax_cos_8175,horaPresMin_varias_8175,horaPresMin_minutos_8175,horaPresMin_sin_8175,horaPresMin_cos_8175,prec_inapreciable_8175,prec_valor_8175,prec_log_8175,dir_sin_8175,dir_cos_8175
0,2014-11-03 14:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
1,2014-11-03 15:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
2,2014-11-03 16:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
3,2014-11-03 17:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
4,2014-11-03 18:00:00,0.0,2014-11-03,3196.0,690.0,11.5,8.0,15.0,6.1,17.5,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648


In [9]:
# Importar librerías necesarias
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Preparar los datos
X = df_precio.drop(['€/kwh'], axis=1)

# Eliminar columnas datetime y object
datetime_object_columns = X.select_dtypes(include=['datetime64[ns]', 'object']).columns
X = X.drop(datetime_object_columns, axis=1)

# Asegurarse de que solo quedan columnas numéricas
X = X.select_dtypes(include=['int64', 'float64', 'bool'])

# Target variable
y = df_precio['€/kwh']

In [ ]:
def evaluate_timepoints(X, y, n_validation_days=3, step_size=30):
    results = []
    
    # Definir puntos de validación
    # Empezamos con suficientes datos para el primer entrenamiento (ej: 6 meses)
    dias = 180
    start_idx = dias * 24

    horas_totales = len(X)

    while start_idx + n_validation_days < len(X):
        # Datos de entrenamiento hasta el punto actual
        X_train = X.iloc[:start_idx]
        y_train = y.iloc[:start_idx]
        
        # Datos de validación (próximos 3 días)
        X_val = X.iloc[start_idx:start_idx + n_validation_days*24]
        y_val = y.iloc[start_idx:start_idx + n_validation_days*24]
        
        # Entrenar modelo
        model = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_val)
        
        # Calcular métricas
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        results.append({
            'fecha_validacion': X_val.index[0],
            'rmse': np.sqrt(mse),
            'r2': r2,
            'y_real': y_val,
            'y_pred': y_pred
        })
        
        print(f"porcentaje entrenado {(start_idx/horas_totales)*100} - RMSE: {np.sqrt(mse)} - R2: {r2}")

        # Avanzar al siguiente punto
        start_idx += step_size
    
    return pd.DataFrame(results)

In [11]:
# Ejecutar evaluación
resultados = evaluate_timepoints(X, y)


Fecha de validación: 180 - RMSE: 0.02275142912943486 - R2: -1.132824804648072
Fecha de validación: 210 - RMSE: 0.042015899665794804 - R2: -0.2106243369052243
Fecha de validación: 240 - RMSE: 0.09431702977821547 - R2: -0.06264087730884671
Fecha de validación: 270 - RMSE: 0.054212274961118655 - R2: -1.0208933316136304
Fecha de validación: 300 - RMSE: 0.03981239441948207 - R2: -11.966427422509742
Fecha de validación: 330 - RMSE: 0.04194450816944948 - R2: -0.20651375704777863
Fecha de validación: 360 - RMSE: 0.05474803594172141 - R2: -0.07619456596926355
Fecha de validación: 390 - RMSE: 0.06536693304734068 - R2: -1.9380849166984682
Fecha de validación: 420 - RMSE: 0.04696111656967775 - R2: -8.5447112454658
Fecha de validación: 450 - RMSE: 0.05121038427563854 - R2: -21.86027180943739
Fecha de validación: 480 - RMSE: 0.0727536871229812 - R2: -4.320383215656621
Fecha de validación: 510 - RMSE: 0.0919114826610247 - R2: -0.3917724418349393
Fecha de validación: 540 - RMSE: 0.037953505421233734 -

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Subplot para RMSE
plt.subplot(2, 1, 1)
plt.plot(resultados['fecha_validacion'], resultados['rmse'])
plt.title('RMSE a lo largo del tiempo')
plt.xlabel('Fecha de validación')
plt.ylabel('RMSE')

# Subplot para R2
plt.subplot(2, 1, 2)
plt.plot(resultados['fecha_validacion'], resultados['r2'])
plt.title('R² a lo largo del tiempo')
plt.xlabel('Fecha de validación')
plt.ylabel('R²')

plt.tight_layout()
plt.show()

# Imprimir estadísticas generales
print("\nEstadísticas de validación:")
print(f"RMSE medio: {resultados['rmse'].mean():.4f}")
print(f"R² medio: {resultados['r2'].mean():.4f}")
print(f"RMSE std: {resultados['rmse'].std():.4f}")
print(f"R² std: {resultados['r2'].std():.4f}")